In [1]:
%pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [13]:
import os
from openai import OpenAI

client = OpenAI(
    base_url=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)
print("> Created OpenAI client.")

> Created OpenAI client.


In [ ]:
PROMPT = """
Score how close the reference answer is to the model answer. You will be comparing these two as JSON objects that
contain 2 keys, "extracted_text" and "clause_type". Score 1.0 if they are both the same, 0.5 if one is the same,
and 0.0 if neither are the same.

Return just a floating point score in the form of JSON with a score field containing the resulting score
and a justification field containing an explanation for the score.
""".strip()

USER_PROMPT = """
Reference answer: {"extracted_text": {{item.extracted_text}}, "clause_type": {{item.clause_type}}}

Model answer: {{sample.output_json}}
""".strip()

GRADER = {
    "name": "Score Model Grader",
    "type": "score_model",
    "model": "o3-mini",
    "input": [
        {"role": "developer", "content": PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
    "range": [0.0, 1.0],
    "pass_threshold": 0.5,
}

SCHEMA = {
    "type": "json_schema",
    "json_schema": {
        "name": "contract_clause",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "clause_type": {
                    "type": "string",
                    "description": "The type of contract clause",
                    "enum": ["Exclusivity", "Non-Compete"],
                },
                "extracted_text": {
                    "type": "string",
                    "description": "The extracted text of the contract clause",
                },
            },
            "required": ["clause_type", "extracted_text"],
            "additionalProperties": False,
        },
    },
}

In [7]:
with open("./clause_matching_training.jsonl", mode="rb") as f:
    training_file = client.files.create(file=f, purpose="fine-tune")
    training_file = client.files.wait_for_processing(training_file.id)

with open("./clause_matching_validation.jsonl", mode="rb") as f:
    validation_file = client.files.create(file=f, purpose="fine-tune")
    validation_file = client.files.wait_for_processing(validation_file.id)

print(f"> created files {training_file.id} and {validation_file.id}")

> created files file-29d025cf762d4836be540750822487e2 and file-541a4f941ad54c1f88c33c252ccbc5de


In [ ]:
job = client.fine_tuning.jobs.create(
    suffix="dv-clause-test",
    model="gpt-5-2025-08-07",
    training_file=training_file.id,
    validation_file=validation_file.id,
    method={
        "type": "reinforcement",
        "reinforcement": {
            "grader": GRADER,
            "response_format": SCHEMA,
        },
    },
)

print(f"> Created RFT job {job.id}")
print(job.to_json())

BadRequestError: Error code: 400 - {'error': {'code': 'invalidPayload', 'message': 'The fineTuningJob field is required.'}}

In [19]:
for page in client.fine_tuning.jobs.list().iter_pages():
    for job in page.data:
        print(f"> job {job.id} -> {job.model}\t\t{job.suffix}\t\t[{job.status}]")

> job ftjob-80cc84fa48e842d9a8570c469307763d -> gpt-5-2025-08-07		dv-endpoint-countdown-ft-9dddad86		[cancelled]
> job ftjob-d08b3e943f024c90b5ff897f7c25b58a -> o4-mini-2025-04-16		dv-endpoint-countdown-ft-9dddad86		[cancelled]
> job ftjob-284f856ab8b44583bd3f69cce3863e2f -> o4-mini-2025-04-16		dv-endpoint-countdown-ft-40be094c		[cancelled]
> job ftjob-88697a532baf48d5a134c6bd3496be4a -> gpt-4.1-nano-2025-04-14		ai-toolcalling-sft		[succeeded]
> job ftjob-73182e20d9254194ba4b665dae17aecd -> gpt-4.1-nano-2025-04-14		ai-toolcalling-sft		[cancelled]
> job ftjob-df2ee766a883431697bcaecd7349cd20 -> gpt-4.1-nano-2025-04-14		ai-toolcalling-sft		[succeeded]
> job ftjob-fa992ed8ba1f4a378d3d32e9016c6778 -> gpt-4.1-nano-2025-04-14		openbsd-sft		[paused]
> job ftjob-4f9ba7a4109e4cd9a5f06837e4d0fe7d -> gpt-4.1-2025-04-14		openbsd-sft		[succeeded]
> job ftjob-f6dc30082b934953b5839312e6d06636 -> gpt-4o-mini-2024-07-18		dv-global-test		[succeeded]
> job ftjob-94f81621d3b14e868297360116ff5efb -> o4-min

AttributeError: 'FineTuningJob' object has no attribute 'suffix'